In [1]:
# Model Training
from datasets import load_from_disk

datasets = load_from_disk("./encoded_data2")
datasets


c:\Users\user\Desktop\QuestionQuestionModel\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'String', 'Algorithm Design', 'Basic Machine Organisation', 'Computer System', 'Data Manipulation and Analysis', 'Data Organisation and Data Control', 'Elementary Web Authoring', 'Health and Ethical Issues', 'Information Processing', 'Intellectual Property', 'Internet Services and Applications', 'Multimedia Elements', 'Networking and Internet Basics', 'Program Development', 'Threats and Security on the Internet', '__index_level_0__'],
        num_rows: 350
    })
    valid: Dataset({
        features: ['Unnamed: 0', 'String', 'Algorithm Design', 'Basic Machine Organisation', 'Computer System', 'Data Manipulation and Analysis', 'Data Organisation and Data Control', 'Elementary Web Authoring', 'Health and Ethical Issues', 'Information Processing', 'Intellectual Property', 'Internet Services and Applications', 'Multimedia Elements', 'Networking and Internet Basics', 'Program Development', 'Threats and Security on the Inte

In [2]:
model_labels = [label for label in datasets['train'].features.keys() if label not in ['Unnamed: 0', 'String', '__index_level_0__']]

In [3]:
model_labels

['Algorithm Design',
 'Basic Machine Organisation',
 'Computer System',
 'Data Manipulation and Analysis',
 'Data Organisation and Data Control',
 'Elementary Web Authoring',
 'Health and Ethical Issues',
 'Information Processing',
 'Intellectual Property',
 'Internet Services and Applications',
 'Multimedia Elements',
 'Networking and Internet Basics',
 'Program Development',
 'Threats and Security on the Internet']

In [4]:
id2label = {idx:label for idx, label in enumerate(model_labels)}
label2id = {label:idx for idx, label in enumerate(model_labels)}

In [5]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import numpy as np

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

In [6]:
def encode_data(dataset):
    text = dataset["String"]
    # tokenize string
    encoding = tokenizer(text, padding="max_length", truncation=True, max_length=70)
    # create encoded array list with labels
    labelsBatch = {x: dataset[x] for x in dataset.keys() if x in model_labels}
    labels_matrix = np.zeros((len(text), len(model_labels)))
    for idx, label in enumerate(model_labels):
        labels_matrix[:, idx] = labelsBatch[label]
    
    encoding["labels"] = labels_matrix.tolist()
    return encoding

In [8]:
encoded_ds = datasets.map(encode_data, batched=True, remove_columns=datasets['train'].column_names)

Loading cached processed dataset at c:\Users\user\Desktop\QuestionQuestionModel\model\encoded_data2\train\cache-2b16d6c5bd35f15f.arrow
Loading cached processed dataset at c:\Users\user\Desktop\QuestionQuestionModel\model\encoded_data2\valid\cache-689012c425206285.arrow
Loading cached processed dataset at c:\Users\user\Desktop\QuestionQuestionModel\model\encoded_data2\test\cache-85c90415b3ef70cb.arrow


In [9]:
encoded_ds['train'][0].keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [10]:
[id2label[idx] for idx, label in enumerate(encoded_ds['train'][15]['labels']) if label == 1.0]

['Networking and Internet Basics']

In [11]:
import torch
from torch import cuda
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

encoded_ds.set_format("torch")

Using device: cuda



In [12]:
len(model_labels)

14

In [13]:

# define model
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", problem_type="multi_label_classification", num_labels=len(model_labels), id2label=id2label, label2id=label2id)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.w

In [31]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(encoded_ds["train"], batch_size=8)
eval_dataloader = DataLoader(encoded_ds["valid"], batch_size=8)

In [14]:
# train model
batch_size = 8
metric_name = "f1"

In [15]:
import evaluate

metric  = evaluate.load("accuracy")

In [16]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    output_dir="./distBert/trainer",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs= 70,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

In [17]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
    
def multi_label_metrics(predictions, labels, threshold=0.5):
    # apply sigmoid on predictions fitting (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # logits, labels = predictions
    # prediction = np.argmax(logits, axis=-1)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

def compute_metrics_new(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [18]:
encoded_ds['train'][0]['labels'].type()

'torch.FloatTensor'

In [19]:
# model.to(device)

In [20]:
outputs = model(input_ids=encoded_ds['train']['input_ids'][0].unsqueeze(0), labels=encoded_ds['train'][0]['labels'].unsqueeze(0))
outputs

SequenceClassifierOutput(loss=tensor(0.7083, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[-0.0888, -0.0467,  0.0785,  0.0748,  0.1638, -0.0354,  0.0353, -0.0033,
          0.1245, -0.0093,  0.0845,  0.0318,  0.0856,  0.0754]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [21]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_ds["train"],
    eval_dataset=encoded_ds["valid"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

In [22]:
trainer.train()

c:\Users\user\Desktop\QuestionQuestionModel\venv\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 350
  Num Epochs = 70
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3080
  Number of trainable parameters = 66964238
  1%|▏         | 42/3080 [00:03<02:34, 19.65it/s] ***** Running Evaluation *****
  Num examples = 44
  Batch size = 8
                                                 
  1%|▏         | 44/3080 [00:03<02:34, 19.65it/s]Saving model checkpoint to ./distBert/trainer\checkpoint-44
Configuration saved in ./distBert/trainer\checkpoint-44\config.json


{'eval_loss': 0.3572169542312622, 'eval_f1': 0.0, 'eval_roc_auc': 0.5, 'eval_accuracy': 0.0, 'eval_runtime': 0.0798, 'eval_samples_per_second': 551.564, 'eval_steps_per_second': 75.213, 'epoch': 1.0}


Model weights saved in ./distBert/trainer\checkpoint-44\pytorch_model.bin
tokenizer config file saved in ./distBert/trainer\checkpoint-44\tokenizer_config.json
Special tokens file saved in ./distBert/trainer\checkpoint-44\special_tokens_map.json
  3%|▎         | 87/3080 [00:06<02:30, 19.85it/s]***** Running Evaluation *****
  Num examples = 44
  Batch size = 8
                                                 
  3%|▎         | 88/3080 [00:06<02:30, 19.85it/s]Saving model checkpoint to ./distBert/trainer\checkpoint-88
Configuration saved in ./distBert/trainer\checkpoint-88\config.json


{'eval_loss': 0.28318551182746887, 'eval_f1': 0.0, 'eval_roc_auc': 0.5, 'eval_accuracy': 0.0, 'eval_runtime': 0.0766, 'eval_samples_per_second': 574.044, 'eval_steps_per_second': 78.279, 'epoch': 2.0}


Model weights saved in ./distBert/trainer\checkpoint-88\pytorch_model.bin
tokenizer config file saved in ./distBert/trainer\checkpoint-88\tokenizer_config.json
Special tokens file saved in ./distBert/trainer\checkpoint-88\special_tokens_map.json
  4%|▍         | 131/3080 [00:10<02:27, 19.99it/s]***** Running Evaluation *****
  Num examples = 44
  Batch size = 8
                                                  
  4%|▍         | 132/3080 [00:10<02:27, 19.99it/s]Saving model checkpoint to ./distBert/trainer\checkpoint-132
Configuration saved in ./distBert/trainer\checkpoint-132\config.json


...

Model weights saved in ./distBert/trainer\checkpoint-3036\pytorch_model.bin
tokenizer config file saved in ./distBert/trainer\checkpoint-3036\tokenizer_config.json
Special tokens file saved in ./distBert/trainer\checkpoint-3036\special_tokens_map.json
100%|█████████▉| 3078/3080 [08:30<00:00, 19.44it/s]***** Running Evaluation *****
  Num examples = 44
  Batch size = 8
                                                   
100%|██████████| 3080/3080 [08:30<00:00, 19.44it/s]Saving model checkpoint to ./distBert/trainer\checkpoint-3080
Configuration saved in ./distBert/trainer\checkpoint-3080\config.json


{'eval_loss': 0.11801730841398239, 'eval_f1': 0.7640449438202247, 'eval_roc_auc': 0.8480046948356809, 'eval_accuracy': 0.5909090909090909, 'eval_runtime': 0.0813, 'eval_samples_per_second': 541.458, 'eval_steps_per_second': 73.835, 'epoch': 70.0}


100%|██████████| 3080/3080 [08:40<00:00, 19.44it/s]Model weights saved in ./distBert/trainer\checkpoint-3080\pytorch_model.bin
tokenizer config file saved in ./distBert/trainer\checkpoint-3080\tokenizer_config.json
Special tokens file saved in ./distBert/trainer\checkpoint-3080\special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./distBert/trainer\checkpoint-1100 (score: 0.7865168539325843).
100%|██████████| 3080/3080 [08:42<00:00,  5.90it/s]

{'train_runtime': 522.3716, 'train_samples_per_second': 46.901, 'train_steps_per_second': 5.896, 'train_loss': 0.045695906645291814, 'epoch': 70.0}


TrainOutput(global_step=3080, training_loss=0.045695906645291814, metrics={'train_runtime': 522.3716, 'train_samples_per_second': 46.901, 'train_steps_per_second': 5.896, 'train_loss': 0.045695906645291814, 'epoch': 70.0})

In [30]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 44
  Batch size = 8
100%|██████████| 6/6 [00:00<00:00, 90.93it/s]


{'eval_loss': 0.10055262595415115,
 'eval_f1': 0.7865168539325843,
 'eval_roc_auc': 0.8593016431924883,
 'eval_accuracy': 0.6136363636363636,
 'eval_runtime': 0.1896,
 'eval_samples_per_second': 232.076,
 'eval_steps_per_second': 31.647,
 'epoch': 70.0}

In [28]:
training_results = trainer.evaluate(encoded_ds['train'])

***** Running Evaluation *****
  Num examples = 350
  Batch size = 8
100%|██████████| 44/44 [00:00<00:00, 81.60it/s]


In [25]:
prediction_result = trainer.predict(encoded_ds['test'])

***** Running Prediction *****
  Num examples = 44
  Batch size = 8
100%|██████████| 6/6 [00:00<00:00, 88.25it/s]


In [26]:
print(prediction_result.metrics)

{'test_loss': 0.09050417691469193, 'test_f1': 0.8139534883720931, 'test_roc_auc': 0.8619424882629108, 'test_accuracy': 0.6818181818181818, 'test_runtime': 0.2745, 'test_samples_per_second': 160.285, 'test_steps_per_second': 21.857}


In [32]:
trainer.save_model("./distlBert-model-v1")

Saving model checkpoint to ./distlBert-model-v1
Configuration saved in ./distlBert-model-v1\config.json
Model weights saved in ./distlBert-model-v1\pytorch_model.bin
tokenizer config file saved in ./distlBert-model-v1\tokenizer_config.json
Special tokens file saved in ./distlBert-model-v1\special_tokens_map.json


In [ ]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('distlbert/')